In [1]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available

True


In [2]:
!pip install llama-cpp-python diffusers torch torchvision transformers accelerate numpy pillow opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install llama-cpp-python torch diffusers transformers accelerate safetensors matplotlib Pillow gradio

In [1]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
from google.colab import drive

# Load the fine-tuned model
def load_fine_tuned_model(model_path=None):
    if not model_path:
        model_path = '/content/drive/MyDrive/fine_tuned_tiny_stories_model'

    try:
        print(f"Loading fine-tuned model from {model_path}...")
        model = AutoModelForCausalLM.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        print(f"Model loaded successfully and moved to {device}")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model: {e}")
        # Try loading a fallback model if the fine-tuned one fails
        try:
            print("Attempting to load fallback model gpt2...")
            model = AutoModelForCausalLM.from_pretrained("gpt2")
            tokenizer = AutoTokenizer.from_pretrained("gpt2")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            print(f"Fallback model loaded successfully and moved to {device}")
            return model, tokenizer
        except Exception as e2:
            print(f"Error loading fallback model: {e2}")
            return None, None

# Helper functions for story generation
def extract_character_name(prompt):
    character_names = ["Spider-Man", "Peter Parker", "Batman", "Superman", "Wonder Woman",
                       "Iron Man", "Captain America", "Thor", "Hulk", "Black Widow"]

    for name in character_names:
        if name.lower() in prompt.lower():
            return name

    words = prompt.split()
    for word in words:
        if word[0].isupper() and len(word) > 1 and word.lower() not in ["write", "story", "about"]:
            return word

    return None

def extract_topic(prompt):
    cleaned = prompt.lower().replace("write", "").replace("story", "").replace("about", "")
    keywords = ["adventure", "flying", "jumping", "fighting", "learning", "playing", "making friends"]

    for keyword in keywords:
        if keyword in cleaned:
            return keyword

    return cleaned.strip()

def generate_story(model, tokenizer, prompt, max_length=400, retries=3):
    story_prompt = f"Write a simple children's story about {prompt}. Use simple language and short sentences."

    # Increase repetition penalty and adjust parameters for better generation
    for attempt in range(retries):
        try:
            input_ids = tokenizer.encode(story_prompt, return_tensors='pt')
            device = next(model.parameters()).device
            input_ids = input_ids.to(device)

            # Use different generation parameters for each retry attempt
            temperature = 0.7 + (attempt * 0.1)  # Gradually increase temperature with each retry
            repetition_penalty = 1.2 + (attempt * 0.1)  # Gradually increase repetition penalty

            output = model.generate(
                input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=3,  # Increased from 2
                top_k=50,  # Increased from 40
                top_p=0.92,  # Adjusted from 0.9
                temperature=temperature,
                do_sample=True,
                repetition_penalty=repetition_penalty
            )

            generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

            # Clean up the generated text to focus on the actual story
            if story_prompt in generated_text:
                generated_text = generated_text.replace(story_prompt, "").strip()

            # Check if we have a reasonable story
            if len(generated_text) > 100 and "." in generated_text:
                print(f"Successfully generated story on attempt {attempt+1}")
                return generated_text
            else:
                print(f"Attempt {attempt+1} produced insufficient content ({len(generated_text)} chars). Retrying...")

        except Exception as e:
            print(f"Error in story generation attempt {attempt+1}: {e}")

    # If all retries fail, generate a more varied story using a different approach
    print("All generation attempts failed. Using alternative generation method...")
    return generate_alternative_story(prompt)

def generate_alternative_story(prompt):
    """
    Generate a more varied story when the model-based generation fails.
    This creates a more dynamic story than the previous structured approach.
    """
    character = extract_character_name(prompt) or "the little hero"
    topic = extract_topic(prompt) or "having an adventure"

    # Create a list of possible story templates
    story_templates = [
        f"""Once upon a time, there was {character} who lived in a small village.
{character} loved {topic} more than anything else.

One day, {character} discovered something magical while playing outside. It was a tiny glowing ball that could float in the air.

"What's this?" wondered {character}. The ball started to move, and {character} followed it into the forest.

The ball led {character} to a hidden garden where other children were playing with similar magical balls. They welcomed {character} and taught them new games.

{character} had so much fun that day! When it was time to go home, one of the children gave {character} a magical ball of their own.

From that day on, {character} visited the hidden garden often and made many new friends. They all loved {topic} together and had wonderful adventures.""",

        f"""{character} woke up early one morning with a big smile. Today was the day for {topic}!

{character} packed a small bag with snacks and a water bottle. "I'm ready for my adventure!" {character} said.

Outside, {character} met a friendly cat. "Hello, cat! Would you like to join me for {topic}?" The cat meowed and followed along.

As they walked through the park, they found a lost teddy bear. "We should find who this belongs to," said {character}.

They asked everyone they met about the teddy bear. Finally, they met a little girl who was crying. "My teddy!" she said happily when she saw it.

"Thank you for helping me find my teddy," the girl smiled. "Would you like to play with me?"

{character}, the cat, and the girl spent the whole day playing and having fun with {topic}. It was the best day ever!""",

        f"""The sun was shining brightly when {character} decided it was the perfect day for {topic}.

"I've never tried {topic} before," {character} thought, "but today I will be brave!"

{character} began slowly, taking small steps. It wasn't easy at first, and {character} almost gave up.

"Don't worry, you can do it!" said a friendly voice. {character} looked up and saw a smiling butterfly.

With the butterfly's encouragement, {character} tried again. This time, {character} did much better!

Soon, {character} was having so much fun with {topic}. "Thank you for believing in me," {character} said to the butterfly.

The butterfly fluttered around happily. "Sometimes we just need a friend to help us be brave," it replied.

{character} went home that evening with a happy heart, excited to try {topic} again tomorrow."""
    ]

    import random
    return random.choice(story_templates)

def split_story_into_panels(story, num_panels=4):
    if len(story) < 100:
        print("Warning: Story is very short. Panel division may not be optimal.")

    paragraphs = story.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]

    dialogues = []
    panel_texts = []

    if len(paragraphs) >= num_panels:
        if len(paragraphs) > num_panels:
            step = len(paragraphs) / num_panels
            panels = []
            for i in range(num_panels):
                start_idx = int(i * step)
                end_idx = int((i + 1) * step) if i < num_panels - 1 else len(paragraphs)
                panel_text = " ".join(paragraphs[start_idx:end_idx])
                panels.append(panel_text)

                dialogue = extract_dialogue(panel_text)
                dialogues.append(dialogue)

                clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues
        else:
            panels = paragraphs[:num_panels]
            for panel in panels:
                dialogue = extract_dialogue(panel)
                dialogues.append(dialogue)
                clean_text = panel.replace(dialogue, "") if dialogue else panel
                panel_texts.append(clean_text)
            return panels, panel_texts, dialogues

    sentences = []
    for chunk in story.split('. '):
        sentences.extend([s.strip() + '.' for s in chunk.split('! ')])
    sentences = [s for s in sentences if len(s) > 1]

    if len(sentences) < num_panels:
        while len(sentences) < num_panels:
            longest_idx = max(range(len(sentences)), key=lambda i: len(sentences[i]))
            longest = sentences[longest_idx]

            if ', ' in longest:
                parts = longest.split(', ', 1)
                sentences[longest_idx] = parts[0] + '.'
                sentences.insert(longest_idx + 1, parts[1])
            else:
                sentences.append(f"Meanwhile, {longest.lower()}")

    step = max(1, len(sentences) // num_panels)

    panels = []
    for i in range(num_panels):
        start_idx = i * step
        end_idx = (i + 1) * step if i < num_panels - 1 else len(sentences)
        panel_text = " ".join(sentences[start_idx:end_idx])
        panels.append(panel_text)

        dialogue = extract_dialogue(panel_text)
        dialogues.append(dialogue)

        clean_text = panel_text.replace(dialogue, "") if dialogue else panel_text
        panel_texts.append(clean_text)

    return panels, panel_texts, dialogues

def extract_dialogue(text):
    import re
    dialogue_matches = re.findall(r'"([^"]*)"', text)
    if dialogue_matches:
        return dialogue_matches[0]
    return ""

def load_custom_model():
    print("Loading InkoloRA model - this may take a while...")

    custom_model_path = "/content/drive/MyDrive/InkoloRA.safetensors"

    if not os.path.exists(custom_model_path):
        print(f"Warning: Custom model not found at {custom_model_path}. Falling back to Stable Diffusion XL.")
        return load_sdxl_model()

    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            torch_dtype=torch.float16,
            safety_checker=None
        )

        pipe.unet.load_attn_procs(custom_model_path)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        if device == "cpu":
            print("Warning: CUDA not available, using CPU. This will be very slow.")
        pipe = pipe.to(device)

        print("InkoloRA model loaded successfully!")
        return pipe
    except Exception as e:
        print(f"Error loading custom model: {e}")
        print("Falling back to Stable Diffusion XL...")
        return load_sdxl_model()

def load_sdxl_model():
    print("Loading SDXL model - this may take a while...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("Warning: CUDA not available, using CPU. This will be very slow.")
    pipe = pipe.to(device)

    return pipe

def add_speech_bubble(image, text, position=None):
    if not text:
        return image

    img = np.array(image.convert("RGBA"))
    height, width, _ = img.shape

    if position is None:
        position = (width // 2, height // 4)

    bubble = np.zeros((height, width, 4), dtype=np.uint8)

    bubble_width = min(300, max(150, len(text) * 10))
    bubble_height = min(130, max(60, (len(text) // 20 + 1) * 30))

    cv2.ellipse(bubble, position, (bubble_width, bubble_height), 0, 0, 360, (255, 255, 255, 255), -1)

    pointer_x = position[0]
    pointer_y = position[1] + bubble_height
    triangle_pts = np.array([
        [pointer_x - 20, pointer_y - 10],
        [pointer_x + 20, pointer_y - 10],
        [pointer_x, pointer_y + 30]
    ])
    cv2.fillPoly(bubble, [triangle_pts], (255, 255, 255, 255))

    bubble_pil = Image.fromarray(bubble)

    draw = ImageDraw.Draw(bubble_pil)

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
        except:
            font = ImageFont.load_default()

    wrapped_text = wrap_text(text, font, bubble_width - 40)

    text_x = position[0] - (bubble_width // 2) + 20
    text_y = position[1] - (bubble_height // 2) + 10

    for i, line in enumerate(wrapped_text):
        draw.text((text_x, text_y + i * 30), line, fill="black", font=font)

    return Image.alpha_composite(image.convert("RGBA"), bubble_pil)

def wrap_text(text, font, max_width):
    words = text.split()
    wrapped_lines = []
    current_line = []

    for word in words:
        test_line = ' '.join(current_line + [word])
        try:
            try:
                bbox = font.getbbox(test_line)
                line_width = bbox[2] - bbox[0]
            except AttributeError:
                try:
                    line_width = font.getlength(test_line)
                except AttributeError:
                    line_width = font.getsize(test_line)[0]
        except:
            line_width = len(test_line) * 10

        if line_width <= max_width:
            current_line.append(word)
        else:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return wrapped_lines

def get_template_data(template_name):
    templates = {
        "batman": {
            "prompts": [
                "Batman standing on a dark Gotham rooftop, his cape flowing in the wind, the Bat-Signal glowing in the sky behind him.",
                "Close-up shot of Batman's determined face, shadows partially covering his mask, eyes glowing in the darkness.",
                "Batman leaping from a rooftop, his cape spread like wings as he descends towards a group of criminals.",
                "Batman delivering a powerful punch to a thug, the action captured in a comic-style motion blur."
            ],
            "dialogues": [
                "Gotham never sleeps, and neither do I.",
                "Justice isn't a choice. It's a responsibility.",
                "Fear is a tool. I use it against them.",
                "Crime has no place in my city."
            ],
            "positions": [(250, 70), (300, 50), (220, 120), (280, 140)],
            "story": """The Dark Knight watches over Gotham City from high above. Tonight is like any other night for Batman - a city full of shadows that need his protection.

As the Bat-Signal illuminates the cloudy sky, Batman's eyes narrow with determination. The signal isn't just a call for help - it's a reminder of the oath he took to protect the innocent.

Batman spots a group of criminals attempting to rob a local shop. He leaps from the rooftop, his cape billowing around him like massive wings. The criminals look up in terror as his silhouette blots out the moonlight.

With swift, calculated movements, Batman takes down the criminals one by one. Another night in Gotham, another battle won. But Batman knows his work is never truly done. As long as there is crime in Gotham, he will be there to fight it."""
        },
        "spiderman": {
            "prompts": [
                "Spider-Man perched on the edge of a skyscraper, overlooking New York City, his iconic red and blue suit vibrant against the skyline.",
                "Spider-Man shooting webs from his wrists, swinging between buildings with great agility.",
                "Spider-Man facing off against a villain, in a dynamic action pose, ready to fight.",
                "Spider-Man helping a civilian, showcasing his friendly neighborhood hero nature."
            ],
            "dialogues": [
                "With great power comes great responsibility.",
                "Just your friendly neighborhood Spider-Man!",
                "My spider-sense is tingling!",
                "I'm not going to let anyone get hurt on my watch."
            ],
            "positions": [(250, 70), (280, 80), (240, 110), (260, 90)],
            "story": """Peter Parker sits perched on the edge of a towering skyscraper, overlooking the bustling streets of New York City. His uncle Ben's words echo in his mind as he surveys the city he's sworn to protect.

With a flick of his wrist, Spider-Man launches himself off the building, shooting a web that attaches to a nearby structure. He swings gracefully through the concrete canyons of Manhattan, the wind rushing past his mask as pedestrians below point and cheer.

Suddenly, his spider-sense alerts him to danger. A bank robbery is in progress just a few blocks away. Spider-Man changes direction mid-swing, heading straight for the trouble. As he arrives, he sees masked robbers threatening innocent civilians.

With swift movements and well-placed webs, Spider-Man subdues the criminals and ensures everyone is safe. A young child approaches him with wide eyes of admiration. Spider-Man kneels down to their level, reminding them that anyone can be a hero if they choose to help others."""
        },
        "wonderwoman": {
            "prompts": [
                "Wonder Woman standing heroically, her golden lasso and bracelets gleaming, an island paradise visible in the background.",
                "Wonder Woman deflecting bullets with her bracelets, her expression determined and fierce.",
                "Wonder Woman wielding her sword and shield, engaged in an epic battle against mythological creatures.",
                "Wonder Woman using her lasso of truth, the golden rope glowing with magical energy."
            ],
            "dialogues": [
                "I am Diana of Themyscira, daughter of Hippolyta.",
                "I fight for those who cannot fight for themselves.",
                "Peace is a virtue to be shown, not peace enforced.",
                "The truth will set you free."
            ],
            "positions": [(250, 70), (270, 60), (230, 100), (260, 80)],
            "story": """The sun rises over Themyscira, casting a golden glow on the island paradise. Diana stands proudly on a cliff overlooking the ocean, her armor gleaming in the morning light, ready to face whatever challenges the day may bring.

News reaches the Amazon princess of danger in the world of men. Without hesitation, Diana leaves her homeland behind. In the midst of chaos, Wonder Woman stands firm, her bracelets deflecting a barrage of bullets meant to harm innocent bystanders.

Ancient mythological creatures, released by a power-hungry sorcerer, terrorize a small village. Wonder Woman meets them in battle, her sword and shield moving with incredible speed and precision. Her training as an Amazon warrior guides each strike.

With the creatures defeated, Wonder Woman confronts the sorcerer. She wraps her golden Lasso of Truth around him, the mystical rope glowing with divine energy. As the magic of the lasso compels him to speak honestly, the truth behind his actions is revealed, and justice can finally be served."""
        }
    }

    return templates.get(template_name.lower(), None)

def get_available_templates():
    return ["Batman", "Spiderman", "WonderWoman"]

def generate_template_comic(template_name, num_panels=4):
    template_data = get_template_data(template_name.lower())

    if not template_data:
        print(f"Template '{template_name}' not found. Available templates: {', '.join(get_available_templates())}")
        return None, None, None, None

    try:
        pipe = load_sdxl_model()

        print(f"\nGenerating {template_name} comic panels...")
        panel_images = []

        print("Generating base character image...")
        base_prompt = f"A detailed illustration of {template_name} in a dynamic pose, full body visible"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        prompts = template_data["prompts"][:num_panels]
        dialogues = template_data["dialogues"][:num_panels]
        positions = template_data["positions"][:num_panels]

        while len(prompts) < num_panels:
            prompts.append(f"{template_name} in an action scene")
        while len(dialogues) < num_panels:
            dialogues.append("")
        while len(positions) < num_panels:
            positions.append((256, 100))

        for i, prompt in enumerate(prompts):
            print(f"Generating image for panel {i+1}...")

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.65,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                image = add_speech_bubble(image, dialogues[i], positions[i])

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save(f"{template_name}_comic_strip.png")
        print(f"{template_name} comic generation complete!")

        # Use the pre-written story from the template data
        story_text = template_data["story"]
        story_panels = prompts

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error generating template comic: {e}")
        return None, None, None, None

def generate_comic(story_prompt, num_panels=4, model_path=None, art_style="cartoon style", use_template=None):
    if use_template and use_template.lower() != "none":
        return generate_template_comic(use_template, num_panels)

    if len(story_prompt) < 10:
        story_prompt = f"Create an interesting story about {story_prompt}"

    try:
        print("Loading fine-tuned model...")
        model, tokenizer = load_fine_tuned_model(model_path)

        if model is None or tokenizer is None:
            raise Exception("Failed to load the fine-tuned model")

        print(f"Generating story from prompt: '{story_prompt}'")
        story_text = generate_story(model, tokenizer, story_prompt)
        print("Story generated:")
        print(story_text)

        # If story generation still produces insufficient content after multiple attempts
        # and with different parameters, try generating with a completely different approach
        if not story_text or len(story_text) < 50:
            print("Story generation produced insufficient content. Using alternative generation approach.")
            story_text = generate_alternative_story(story_prompt)
            print("Alternative story generated:")
            print(story_text)

    except Exception as e:
        print(f"Error in model-based story generation: {e}")
        print("Using alternative story generation approach...")
        story_text = generate_alternative_story(story_prompt)

    print("\nSplitting story into panels...")
    story_panels, panel_texts, dialogues = split_story_into_panels(story_text, num_panels)
    for i, panel in enumerate(story_panels):
        print(f"Panel {i+1}: {panel}")
        if dialogues[i]:
            print(f"Dialogue for panel {i+1}: \"{dialogues[i]}\"")

    try:
        pipe = load_custom_model()

        print("\nGenerating panel images...")
        panel_images = []

        character_name = extract_character_name(story_prompt) or "the character"

        print("Generating base character image for consistency...")
        base_prompt = f"A friendly {art_style} illustration of {character_name}, whole body visible, simple background, bright colors, child-friendly"

        is_sdxl = isinstance(pipe, StableDiffusionXLPipeline)

        if is_sdxl:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30
            ).images[0]
        else:
            base_image = pipe(
                prompt=base_prompt,
                negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                num_inference_steps=30,
                guidance_scale=7.5
            ).images[0]

        base_image = base_image.resize((768, 768))

        for i, panel_text in enumerate(panel_texts):
            print(f"Generating image for panel {i+1}...")
            # Make the image prompts unique for each panel based on the actual story content
            prompt = f"A friendly {art_style} illustration for a children's storybook showing: {panel_text}, featuring {character_name}, bright colors, simple shapes, cute characters, child-friendly"

            if is_sdxl:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30
                ).images[0]
            else:
                image = pipe(
                    prompt=prompt,
                    image=base_image,
                    strength=0.7,
                    negative_prompt="blurry, low quality, text, watermark, signature, bad proportions, scary, dark, adult themes",
                    num_inference_steps=30,
                    guidance_scale=7.5
                ).images[0]

            image = image.resize((512, 512))

            if dialogues[i]:
                position = (256, 100 + (i % 2) * 50)
                image = add_speech_bubble(image, dialogues[i], position)

            panel_images.append(image)

            image_path = f"panel_{i+1}.png"
            image.save(image_path)

        print("Creating combined comic strip...")
        rows = int(np.ceil(num_panels / 2))
        columns = min(2, num_panels)

        final_width = 512 * columns
        final_height = 512 * rows

        comic_strip = Image.new("RGB", (final_width, final_height), (255, 255, 255))

        for i, image in enumerate(panel_images):
            x_offset = (i % 2) * 512
            y_offset = (i // 2) * 512
            comic_strip.paste(image, (x_offset, y_offset))

        comic_strip.save("comic_strip.png")
        print("Comic generation complete! All panels saved as PNG files and combined into comic_strip.png")

        return story_text, story_panels, panel_images, comic_strip

    except Exception as e:
        print(f"Error in image generation: {e}")
        print("Unable to generate images. Please check if you have enough GPU memory and proper installations.")
        return story_text, story_panels, None, None

# Web UI using Gradio
def create_ui():
    with gr.Blocks() as app:
        gr.Markdown("# Story to Comic Generator with InkoloRA")
        gr.Markdown("Enter a prompt to generate a story and convert it into comic panels with speech bubbles using the InkoloRA model")

        with gr.Row():
            with gr.Column():
                template_dropdown = gr.Dropdown(
                    choices=["None"] + get_available_templates(),
                    label="Choose Template (optional)",
                    value="None"
                )

                story_prompt = gr.Textbox(
                    label="Story Prompt",
                    placeholder="Enter a prompt for the children's story (e.g. 'a little boy who learns to fly')",
                    lines=2
                )

                num_panels = gr.Slider(
                    label="Number of Panels",
                    minimum=2,
                    maximum=8,
                    value=4,
                    step=1
                )

                art_style = gr.Dropdown(
                    choices=["Cartoon Style", "Realistic", "Anime", "Sketch"],
                    label="Art Style",
                    value="Cartoon Style"
                )

                generate_button = gr.Button("Generate Comic")

            with gr.Column():
                output_text = gr.Textbox(
                    label="Generated Story",
                    placeholder="The generated story will appear here...",
                    lines=10,
                    interactive=False
                )

                output_panels = gr.Gallery(
                    label="Comic Panels",
                    show_label=True,
                    elem_id="comic-panels",
                    height=200,
                    columns=2
                )

                output_comic = gr.Image(
                    label="Combined Comic Strip",
                    type="pil"
                )

        # Fixed function to handle the generation and properly return outputs
        def generate_comic_action(story_prompt, num_panels, art_style, use_template):
            story_text, story_panels, panel_images, comic_strip = generate_comic(
                story_prompt, num_panels, art_style=art_style, use_template=use_template
            )
            # Return the outputs directly rather than updating them
            return story_text, panel_images if panel_images else [], comic_strip

        generate_button.click(
            generate_comic_action,
            inputs=[story_prompt, num_panels, art_style, template_dropdown],
            outputs=[output_text, output_panels, output_comic]
        )

    app.launch()

# Mount Google Drive if in Colab environment
try:
    drive.mount('/content/drive')
except:
    print("Not running in Colab or drive already mounted")

# Call the function to create the UI
if __name__ == "__main__":  # Fixed from 'name' to 'name_'
    create_ui()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82163ff8ae0f07cbf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
